# 3d mitochondria visualizer using vtk 
- Includes a Neuroglancer link generator and a vtk viewer centered on a mitoid of interest 
- This notebook is modified from Allen Institute tutorial on working with Meshes by Forrest Collman  
The original tutorial is located here:   https://github.com/AllenInstitute/MicronsBinder/blob/master/notebooks/intro/MeshExample.ipynb

## Python Requirements
You will need to run this notebook in a Python 3.7 environment  
Sometime meshparty will have a conflict with other tools and not work properly in Anaconda  
If that happens, you will need to start over with a new Anaconda environment  
Install allensdk first, then meshparty, then any other other tools desired  
The installation described below worked well:  
* Install new environment v3.7 in Anaconda
* conda install jupyter
* pip install allensdk
* pip install meshparty
* pip install caveclient
* pip install 'itkwidgets[notebook]>=1.0a8'
* pip install --upgrade --pre itk
* pip install gdown

In [ ]:
import pandas as pd
import numpy as np
import os

In [ ]:
from meshparty import trimesh_io, trimesh_vtk, skeleton, utils

In [ ]:
import vtk

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
# this csv file is 268 MB and too large to place on Github
# in order to run this notebook, you must first download the file and place on your local machine in the /data folder
# the csv file is available here: https://zenodo.org/record/5579388/files/211019_mitochondria_info.csv

mito = pd.read_csv('data/211019_mitochondria_info.csv')
mito

In [ ]:
astrocytes = pd.read_csv('data/astrocytes.csv', index_col=[0])
astrocytes.head()

In [ ]:
astrocyte_list = astrocytes.cell_segid.to_list()
print(astrocyte_list)

# Use index value to view an astrocyte from list 

In [ ]:
# pull all mitos from a cellid of interest from astrocyte list, by index number

indx = 28
cellid = astrocyte_list[indx]

mito_querydf = mito[mito['cellid'] == cellid]
mito_querydf.sort_values(by='mito_vx', ascending=False)


In [ ]:
cellid

In [ ]:
thresh = 2000000

In [ ]:
mito_query_abovethresh = mito_querydf[mito_querydf.mito_vx > thresh]
mito_query_abovethresh

In [ ]:
mito_query_belowthresh = mito_querydf[mito_querydf.mito_vx <= thresh]
mito_query_belowthresh

In [ ]:
mito_query_abovethresh_mitolist = mito_query_abovethresh.mito_id.to_list()
print(f"length: "+str(len(mito_query_abovethresh_mitolist)))
print(f"type: "+str(type(mito_query_abovethresh_mitolist)))
print('')
# print(mito_query_abovethresh_mitolist) # uncomment to print the mito id list

In [ ]:
mito_query_belowthresh_mitolist = mito_query_belowthresh.mito_id.to_list()
print(f"length: "+str(len(mito_query_belowthresh_mitolist)))
print(f"type: "+str(type(mito_query_belowthresh_mitolist)))
print('')
# print(mito_query_belowthresh_mitolist) # uncomment to print the mito id list

# vtk 3d viewer

In [ ]:
# setup the mesh meta to handle downloads and caching
mesh_dir = 'data/neuron_meshes_v185/' # or change to your desired folder
seg_source = "precomputed://https://storage.googleapis.com/microns_public_datasets/pinky100_v185/seg"
mm = trimesh_io.MeshMeta(cv_path=seg_source,
                         disk_cache_path=mesh_dir, 
                         cache_size=20)

# setup the mesh meta to handle downloads and caching
mito_mesh_dir = 'data/meshes/'
mito_source = "precomputed://https://seungdata.princeton.edu/sseung-archive/pinky100-mito/seg_191220"
mito_mm = trimesh_io.MeshMeta(cv_path=mito_source,
                         disk_cache_path=mito_mesh_dir)

In [ ]:
cell_id = cellid

In [ ]:
# If you get an error, run the next cell to download the mesh, then rerun this cell again
seg_id = cell_id
mesh_file = os.path.join(mesh_dir, str(seg_id)+'.h5')
mesh = mm.mesh(filename = mesh_file)

In [ ]:
# If you get an error on the previous cell, run this cell
# After the mesh is successfully downloaded, rerun the previous cell 
downloadmesh = mm.mesh(seg_id = seg_id, remove_duplicate_vertices=True)

In [ ]:
len(mito_query_abovethresh_mitolist)

In [ ]:
len(mito_query_belowthresh_mitolist)

### The following cell only needs to be run once
- You do not need to run this cell if you have previously downloaded the mito mesh files
- Note: this will take a long time, which gets longer the more mitos are in the cell of interest

In [ ]:
# download the mito meshes for this cell

for i in range(len(mito_query_abovethresh_mitolist)):
    mito_id = mito_query_abovethresh_mitolist[i]
    mito_seg_id = mito_id
    mito_downloadmesh = mito_mm.mesh(seg_id = mito_seg_id, remove_duplicate_vertices=True)
    
for i in range(len(mito_query_belowthresh_mitolist)):
    mito_id = mito_query_belowthresh_mitolist[i]
    mito_seg_id = mito_id
    mito_downloadmesh = mito_mm.mesh(seg_id = mito_seg_id, remove_duplicate_vertices=True)


In [ ]:
# iterate the mito_mesh for each mito
# from https://python-forum.io/thread-23500.html

var_iterator_abovethresh = {}
for i in range(len(mito_query_abovethresh_mitolist)):
    mito_seg_id = mito_query_abovethresh_mitolist[i] 
    mito_mesh_file = os.path.join(mito_mesh_dir, str(mito_seg_id)+'.h5')
    var_iterator_abovethresh['mito_mesh_' + str(i)] = mito_mm.mesh(filename = mito_mesh_file)
    
locals().update(var_iterator_abovethresh)


var_iterator_belowthresh = {}
for i in range(len(mito_query_belowthresh_mitolist)):
    mito_seg_id = mito_query_belowthresh_mitolist[i] 
    mito_mesh_file = os.path.join(mito_mesh_dir, str(mito_seg_id)+'.h5')
    var_iterator_belowthresh['mito_mesh_' + str(i)] = mito_mm.mesh(filename = mito_mesh_file)
    
locals().update(var_iterator_belowthresh)


In [ ]:
len(var_iterator_abovethresh)

In [ ]:
var_iterator_abovethresh

In [ ]:
# opacity and color settings for cell membrane and mito meshes
cell_opac = 0.05 # 0.075
cell_color = (1, 1, 1) # (0.2, 0.4, 0.7) blue
mito_opac_abovethresh = 1 # 0.35
mito_color_abovethresh = (0.2, 0.4, 0.7)
mito_opac_belowthresh = 1
mito_color_belowthresh = (0.7, 0.2, 0.2)

# cell membrane mesh
mesh_actor = trimesh_vtk.mesh_actor(mesh, opacity=cell_opac, color=cell_color)

# mito meshes

actor_iterator_abovethresh = {}
for i in range(len(mito_query_abovethresh_mitolist)):
    #random_color = list(np.random.random(size=3))
    actor_iterator_abovethresh['mito_mesh_actor_' + str(i)] = trimesh_vtk.mesh_actor(var_iterator_abovethresh['mito_mesh_'+str(i)], opacity=mito_opac_abovethresh, color=(mito_color_abovethresh))
    
locals().update(actor_iterator_abovethresh)

actor_iterator_belowthresh = {}
for i in range(len(mito_query_belowthresh_mitolist)):
    #random_color = list(np.random.random(size=3))
    actor_iterator_belowthresh['mito_mesh_actor_' + str(i+(len(var_iterator_abovethresh)))] = trimesh_vtk.mesh_actor(var_iterator_belowthresh['mito_mesh_'+str(i)], opacity=mito_opac_belowthresh, color=(mito_color_belowthresh))
    
locals().update(actor_iterator_belowthresh)

# update dictionary for the render actors code below
actor_iterator = {}
actor_iterator['mesh_actor'] = mesh_actor
actor_iterator.update(actor_iterator_abovethresh)
actor_iterator.update(actor_iterator_belowthresh)
locals().update(actor_iterator)

#creating a camera object and defining the view
camera = trimesh_vtk.oriented_camera(mesh.centroid, backoff=200) # 150

#render the actors, will open a pop up python window
trimesh_vtk.render_actors(actor_iterator.values(), camera=camera)


In [ ]:
#actor_iterator

In [ ]:
#type(actor_iterator.values())

In [ ]:
# save image to disk
# uncomment to save

#camera = trimesh_vtk.oriented_camera(mesh.centroid, backoff=190)

#trimesh_vtk.render_actors(actor_iterator.values(),
#                          filename='vtk_images/'+str(cellid)+'.png',
#                          do_save=True,
#                          scale=6,
#                          camera=camera
#                         )

In [ ]:
cellid

In [ ]:
# 648518346349536888 has two to three astrocytes

In [ ]:
# 648518346341354380 very interesting case

In [ ]:
astrocyte_list

# Neuroglancer visualizer

In [ ]:
lookup = cellid

In [ ]:
query = mito[mito["cellid"] == lookup]

In [ ]:
len(query)

In [ ]:
query_list = query["mito_id"].to_list()
spacer = "%22%2C%22"
query_list_join = spacer.join(map(str,query_list))
baseURL = 'https://neuromancer-seung-import.appspot.com/'
nglink_a = '#!%7B%22layers%22:%5B%7B%22source%22:%22precomputed://gs://microns_public_datasets/pinky100_v0/son_of_alignment_v15_rechunked%22%2C%22type%22:%22image%22%2C%22blend%22:%22default%22%2C%22shaderControls%22:%7B%7D%2C%22name%22:%22EM%22%7D%2C%7B%22source%22:%22precomputed://gs://microns_public_datasets/pinky100_v185/seg%22%2C%22type%22:%22segmentation%22%2C%22selectedAlpha%22:0.51%2C%22objectAlpha%22:0.15%2C%22segments%22:%5B%22'
nglink_b = '%22%5D%2C%22skeletonRendering%22:%7B%22mode2d%22:%22lines_and_points%22%2C%22mode3d%22:%22lines%22%7D%2C%22name%22:%22cell_segmentation_v185%22%7D%2C%7B%22source%22:%22precomputed://matrix://sseung-archive/pinky100-clefts/mip1_d2_1175k%22%2C%22type%22:%22segmentation%22%2C%22skeletonRendering%22:%7B%22mode2d%22:%22lines_and_points%22%2C%22mode3d%22:%22lines%22%7D%2C%22name%22:%22synapses%22%7D%2C%7B%22source%22:%22precomputed://matrix://sseung-archive/pinky100-mito/seg_191220%22%2C%22type%22:%22segmentation%22%2C%22segments%22:%5B%22'
nglink_c = '%22%5D%2C%22skeletonRendering%22:%7B%22mode2d%22:%22lines_and_points%22%2C%22mode3d%22:%22lines%22%7D%2C%22name%22:%22mitochondria%22%7D%2C%7B%22source%22:%22precomputed://matrix://sseung-archive/pinky100-nuclei/seg%22%2C%22type%22:%22segmentation%22%2C%22skeletonRendering%22:%7B%22mode2d%22:%22lines_and_points%22%2C%22mode3d%22:%22lines%22%7D%2C%22name%22:%22nuclei%22%7D%5D%2C%22navigation%22:%7B%22pose%22:%7B%22position%22:%7B%22voxelSize%22:%5B4%2C4%2C40%5D%2C%22voxelCoordinates%22:%5B83222.921875%2C52981.34765625%2C824.9962768554688%5D%7D%7D%2C%22zoomFactor%22:383.0066650796121%7D%2C%22perspectiveOrientation%22:%5B-0.00825042650103569%2C0.06130112707614899%2C-0.0012821174459531903%2C0.9980843663215637%5D%2C%22perspectiveZoom%22:3618.7659948513424%2C%22showSlices%22:false%2C%22selectedLayer%22:%7B%22layer%22:%22mitochondria%22%2C%22visible%22:true%7D%2C%22layout%22:%7B%22type%22:%223d%22%2C%22orthographicProjection%22:true%7D%7D'
neuroglancerURL = baseURL + nglink_a + str(lookup) + nglink_b + query_list_join + nglink_c
print(neuroglancerURL)

In [ ]:
query

# Center vtk view at centroid of mitoid of interest

In [ ]:
mitoid_query = 980814

In [ ]:
pull_centroid = mito_querydf.reset_index()
pull_centroid = pull_centroid.index[pull_centroid['mito_id'] == mitoid_query].to_list()
pull_centroid[0]

In [ ]:
mito_mesh_centroid_dict = {}
mito_seg_id = mitoid_query
mito_mesh_file = os.path.join(mito_mesh_dir, str(mito_seg_id)+'.h5')
mito_mesh_centroid_dict['mito_mesh_centroid_' + str(pull_centroid[0])] = mito_mm.mesh(filename = mito_mesh_file)
locals().update(mito_mesh_centroid_dict)

In [ ]:
mito_mesh_centroid_dict

In [ ]:
mito_mesh_centroid_dict.get('mito_mesh_centroid_'+str(pull_centroid[0])).centroid

In [ ]:
# opacity and color settings for cell membrane and mito meshes
cell_opac = 0.05 # 0.075
cell_color = (1, 1, 1) # (0.2, 0.4, 0.7) blue
mito_opac_abovethresh = 1 # 0.35
mito_color_abovethresh = (0.2, 0.4, 0.7)
mito_opac_belowthresh = 1
mito_color_belowthresh = (0.7, 0.2, 0.2)
backoff_value = 20

# cell membrane mesh
mesh_actor = trimesh_vtk.mesh_actor(mesh, opacity=cell_opac, color=cell_color)

# mito meshes

actor_iterator_abovethresh = {}
for i in range(len(mito_query_abovethresh_mitolist)):
    #random_color = list(np.random.random(size=3))
    actor_iterator_abovethresh['mito_mesh_actor_' + str(i)] = trimesh_vtk.mesh_actor(var_iterator_abovethresh['mito_mesh_'+str(i)], opacity=mito_opac_abovethresh, color=(mito_color_abovethresh))
    
locals().update(actor_iterator_abovethresh)

actor_iterator_belowthresh = {}
for i in range(len(mito_query_belowthresh_mitolist)):
    #random_color = list(np.random.random(size=3))
    actor_iterator_belowthresh['mito_mesh_actor_' + str(i+(len(var_iterator_abovethresh)))] = trimesh_vtk.mesh_actor(var_iterator_belowthresh['mito_mesh_'+str(i)], opacity=mito_opac_belowthresh, color=(mito_color_belowthresh))
    
locals().update(actor_iterator_belowthresh)

# update dictionary for the render actors code below
actor_iterator = {}
actor_iterator['mesh_actor'] = mesh_actor
actor_iterator.update(actor_iterator_abovethresh)
actor_iterator.update(actor_iterator_belowthresh)
locals().update(actor_iterator)

#creating a camera object and defining the view
camera = trimesh_vtk.oriented_camera(mito_mesh_centroid_dict.get('mito_mesh_centroid_'+str(pull_centroid[0])).centroid, backoff=backoff_value) # 150

#render the actors, will open a pop up python window
trimesh_vtk.render_actors(actor_iterator.values(), camera=camera)


In [ ]:
# save image to disk
# uncomment to save

camera = trimesh_vtk.oriented_camera(mito_mesh_centroid_dict.get('mito_mesh_centroid_'+str(pull_centroid[0])).centroid, backoff=backoff_value)

trimesh_vtk.render_actors(actor_iterator.values(),
                          filename='vtk_images/'+str(cellid)+'_'+str(mitoid_query)+'.png',
                          do_save=True,
                          scale=6,
                          camera=camera
                         )